# Exercise 1 (3): Image differences

## a (1): Capture images

Write a function to capture a sequence of images using Matlab .
```
Open Matlab and install following packages: here
vid = videoinput('dcam', 1, 'F7_YUV422_812x612_mode1');
vid.FramesPerTrigger = 1;
vid.ReturnedColorspace = 'rgb';
preview(vid);
```
in the command window to create a firewire camera pipeline, then you can capture one frame using:
```
start(vid);
stoppreview(vid);
imwrite(getdata(vid), 'image_path_to_save');
```
where success is a flag that indicates whether the image capture was successful.

Write a function to capture a sequence of images using Python:
```
import cv2
cam = cv2.VideoCapture(0, cv2.CAP_FIREWIRE)
ret_val, img = cam.read()
cv2.imwrite(filename, img)
```

In [1]:
import cv2
import numpy as np

#rgb to bgr
import matplotlib.pyplot as plt

# A = cv2. imread('img.png',0)

# cv2.imshow('windowName', A)
# cam = cv2.VideoCapture(0, cv2.CAP_FIREWIRE)
cam = cv2.VideoCapture(0)
cv2.namedWindow("test")
path = "captureImage/"
img_counter = 0
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)
    k = cv2.waitKey(1)
    if k%256 == 27:
    # ASCII:ESC pressed, exit
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
    # ASCII:SPACE pressed, capture frame
        img_name = path + "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        # print("{} written!".format(img_name))
        img_counter += 1
print("{} written!".format(img_name))
cam.release()
cv2.destroyAllWindows()

Escape hit, closing...
captureImage/opencv_frame_154.png written!


## b (2). Image differences

Compute the temporal image derivatives approximated by successive image differences. In the case of a fixed camera and a moving object (e.g. person walking by), try to reliably threshold the temporal derivative to extract only the moving object.

##### <span style = "color:red;">If you do not use thresholding, what could mistakenly be identified as motion?</span>
<span style = "color:green;">If we do not use thresholding, there is a lot of noise difference added that is not neccesary motion. Shadows are another thing to consider. Without adding the threshold, there can be an increase in a lot of vectors that are technically background in the final video of the optic flow</span>

You need to convert from colorful images to gray scale images before going to next step.




In [4]:
import cv2
import numpy as np
path = "captureImage/"
path1 = "replaceImage/"
ImageShape = [480,640]
imageWidth = ImageShape[1]
imageHeight = ImageShape[0]
size = (imageWidth, imageHeight)
for i in range(10):
    #read 2 images
    img_nameNew = path + "opencv_frame_{}.png".format(i+1)
    img_nameOld = path + "opencv_frame_{}.png".format(i)
    Inew = cv2. imread(img_nameNew,0)
    Iold = cv2. imread(img_nameOld,0) 
    cv2.imshow("test1", Inew)
    cv2.imshow("test2", Iold)
    #convert to signed 16
    Inew = Inew.astype(np.int16)
    Iold = Iold.astype(np.int16)
    dIm = np.absolute(np.array(Inew) - np.array(Iold))
    # take the abolute difference
    dIm = dIm.astype(np.uint8)
    while True:
        # dIm[dIm>=80] = 255
        dIm[dIm<20] = 0
        cv2.imshow("test", dIm)
        k = cv2.waitKey(1)
        if k%256 == 27:
            break
    cv2.imwrite(path1 + "opencv_difference_{}.png".format(i), dIm)
cv2.destroyAllWindows()


## Exercise 2 (7): Optical flow

Solve for the optic flow. Write a routine that outputs the optic flow vectors given an image sequence. Try for both moving camera and moving objects. Chapter 9 from Stockman and Shapiro has a basic presentation of the required concepts: Motion from 2D Image Sequences. Read the introductory sections cursorily. Pay attention to the definitions in Section 9.3 and then closely look at the optic flow equations in Section 9.3.5

Tile the image into blocks of user specified size (try sizes 4x4, 8x8, 16x16)
For each block, consider if there is motion (Use the thresholds you derived above). If no temporal change set motion vector to 0.
If there is temporal change, compute the spatial derivatives and solve for the x and y optic flow vector.
Plot the flow vectors on top of the image using quiver.
Submit a movie of your results (images with overlayed flow field). You can use getframe to create a movie. Please make sure the movie file is relatively small.


#### Example use of quiver and getframe:
```
X = (1 : blockSize : imageWidth-1) + blockSize/2;
Y = (1 : blockSize : imageHeight-1) + blockSize/2;
[X,Y] = meshgrid(X,Y);
loop through all images (i){
  ...
  loop through all blocks (ix,iy){
    compute motion vectors ...
    U(ix,iy) = xMotion;
    V(ix,iy) = yMotion;
  }
  ...
  plot image;
  hold on;
  quiver(X,Y,U,V);
  hold off;
  M(i) = getframe;
}

movie(M);
save myMovie M;
```
If you are getting inconsistent results, you could try testing with artificial motion first. For example:
```
%im1 = image at time 1
%im2 = image at time 2
im2 = zeros(size(im1));
im2(1:end-1, 1:end-1) = im1(2:end, 2:end);
```

You may also find it useful to display some of your intermediate images/data.

##### <span style = "color:red;">Can this optic flow account for any type of motion? If not, give two distinct cases when it will not work well. </span>

<span style = "color:green;">This optic flow only detects the change in pixels in the 2D acces of the plane of the image. It does nont count for any other motion such as rotation, scale, affine, etc. One case it would not work would be if the object was moving closer towards the camera</span>


In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
blockSize = 32
path = "captureImage/"
ImageShape = [480,640]
imageWidth = ImageShape[1]
imageHeight = ImageShape[0]
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
result = cv2.VideoWriter('outputOptic32.mp4', fourcc, 20, (imageWidth, imageHeight))
# result.release()
for index in range(115):
    img_nameNew = path + "opencv_frame_{}.png".format(index+1)
    img_nameOld = path + "opencv_frame_{}.png".format(index)
    Inew = cv2. imread(img_nameNew,0)
    Iold = cv2. imread(img_nameOld,0) 
    size = (imageWidth, imageHeight)

    Inew = Inew.astype(np.float)
    Iold = Iold.astype(np.float)
    # difference image  
    # dIm = np.absolute(np.array(Inew) - np.array(Iold))
    dIm = np.array(Inew) - np.array(Iold)
    dIm = dIm.astype(np.float)
    dIm[np.absolute(dIm)<50 ] = 0
    [dUm, dVm] = np.gradient(Iold)
    #row, column -> y, x
    X = range(1+int(blockSize/2),imageWidth,blockSize)
    Y = range(1+int(blockSize/2),imageHeight,blockSize)
    U = np.ones((len(Y),len(X)))
    V = np.ones((len(Y),len(X)))
    for i in range(len(X)):
        for j in range(len(Y)):
            # get frame
            k = j*blockSize
            l = i*blockSize
            dI = dIm[k:k+blockSize, l:l+blockSize]
            dU = dUm[k:k+blockSize, l:l+blockSize]
            dV = dVm[k:k+blockSize, l:l+blockSize]
            # vector, flatten matrix
            dIv = dI.flatten()
            dUv = dU.flatten()
            dVv = dV.flatten()
            dIv = np.array([dIv])
            #least squares
            M = np.array([dUv, dVv])
            dIv = dIv.T
            M = M.T
            arr = np.linalg.lstsq(M,dIv, rcond=None)[0]
            # normalise
            u, v = -arr / np.linalg.norm(arr)
            #x motion
            U[j,i] = u
            #y motion
            V[j,i] = v

    [X,Y] = np.meshgrid(X,Y)
    Iold = cv2.imread(path + "opencv_frame_{}.png".format(index))
    Iold = cv2.cvtColor(Iold, cv2.COLOR_BGR2RGB)
    fig = plt.figure(frameon = False)
    fig.set_size_inches(16, 12)
    ax = plt.Axes(fig, [0., 0., 1., 1.], )
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.quiver(X,Y,U,V, minlength=4)
    ax.imshow(Iold)
    fig.canvas.draw()
    # matplotlib fig to numpy
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    image_from_plot = cv2.cvtColor(image_from_plot, cv2.COLOR_BGR2RGB)
    image_from_plot = cv2.resize(image_from_plot, (640,480), interpolation = cv2.INTER_AREA)
    # write
    result.write(image_from_plot)
    plt.clf()
    plt.close("all")
print("finished")
result.release()
cv2.destroyAllWindows


<ipython-input-6-af0002725a0a>:52: RuntimeWarning: invalid value encountered in true_divide
  u, v = -arr / np.linalg.norm(arr)


finished


<function destroyAllWindows>

# Exercise 3 (Bonus - 2): High dimensional motion

Try to extract 3-6 dimensional motion. Parameterize motion using e.g. 4 (x,y,rotation,scale) and 6 (affine) parameters. How many motion parameters can be reliably extracted? How does the block size affect the results? Try both camera motion and object motion sequences. Compare results.

Each time you may only be able to show 2dof by using quiver. You can show images two times if you are trying 4 dof, 3 times for 6dof.
While visualizing your extracted motion parameters is difficult, you can also validate your solution by warping the image (or just individual blocks) based on the extracted motion parameters, and checking if the result is more similar to the next frame than if you computed the warped image using your solution to Exercise 2.



In [ ]:
cv2.pydown()
# high degree traking 4 to 6
# pyramid down
#warp, Jap matrix



<center><h1>Lab 1: Optical flow</h1></center>

<center>428/615, Martin Jagersand</center>
<br>
<font color="#FF0000">
Submit code and report electronically through the upload link on the e-Class course webpage.<br>
Marks may be given for any of the questions in the assignment, so
please answer them all <br>(many only require a one sentence answer).<br>
Demo: In your lab session.
<!-- Demo: In your lab session during the week following the due date. -->
</font>
<br>
<br>

<b><font size="+1">Exercise 1 (3):</font></b> <font size="+1">Image differences</font>
<blockquote>
<p><font size="+1">a (1). Capture images</font>

</p><p>Write a function to capture a sequence of images using <code> Matlab </code>.
<br>
Open Matlab and install following packages: <a href="https://www.mathworks.com/hardware-support/dcam.html">here</a>
<br>
<code> vid = videoinput('dcam', 1, 'F7_YUV422_812x612_mode1'); </code>
<br>
<code> vid.FramesPerTrigger = 1; </code>
<br>
<code> vid.ReturnedColorspace = 'rgb'; </code>
<br>
<code> preview(vid); </code>
<br>
in the command window to create a firewire camera pipeline, then you can capture one frame using:
<br>
<code> start(vid);   </code>
<br>
<code> stoppreview(vid);  </code>
<br>
<code> imwrite(getdata(vid), 'image_path_to_save');  </code>
<br>
where <code> success </code> is a flag that indicates whether the image capture was successful.
<br>
</p><p>Write a function to capture a sequence of images using <code> Python </code>.
<br>
<code> import cv2  </code>
<br> 
<code> cam = cv2.VideoCapture(0, cv2.CAP_FIREWIRE)</code>
<br> 
<code> ret_val, img = cam.read() </code>
<br> 
<code> cv2.imwrite(filename, img) </code>
<br> 
<!-- <p>Write a function to capture a sequence of images using <code> xvui2 </code>.
(Open Matlab, type <code> xvui2 </code> in the command window, then you can capture one frame using <code> im = MexVision('frame'); </code>.) -->
 
<!-- Function input should
include number of frames and time between images (this is exactly what 
we did in lab 0 exercise 2). -->


<!-- Use <code> pause </code> to wait between 
images. <em>This function is used for the remaining exercises</em> -- to 
permit testing optic flow and tracking code on a sequence repeatedly. -->

</p><p><font size="+1">b (2). Image differences</font>
</p><p>Compute the temporal image derivatives approximated by successive image 
differences. In the case of a fixed camera and a moving object (e.g. 
person walking by), try to reliably threshold the temporal derivative to 
extract only the moving object.

</p><ul>
<li><strong>If you do not use thresholding, what could mistakenly be identified 
as motion?</strong></li>
<li><strong> You need to convert from colorful images to gray scale images before going
to next step.</strong></li>
</ul>
</blockquote>

<b><font size="+1">Exercise 2 (7):</font></b> <font size="+1">Optical flow</font> 
<blockquote>
<p>Solve for the optic flow. Write a routine that outputs the optic flow 
vectors given an image sequence. Try for both moving camera and moving 
objects. Chapter 9 from Stockman and Shapiro has a basic presentation of 
the required concepts: <a href="http://www.cs.ualberta.ca/~jag/courses/HandEye/MotionReadings/ch9.pdf"> 
Motion from 2D Image 
Sequences</a>. Read the introductory sections cursorily. 
Pay attention to the definitions in Section 9.3 and then closely look at 
the optic flow equations in Section 9.3.5

</p><ul>
<li>Tile the image into blocks of user specified size (try sizes 4x4, 8x8, 
16x16)</li>
<li>For each block, consider if there is motion (Use the thresholds you 
derived above). If no temporal change set motion vector to 0.
</li>
<li>If there is temporal change, compute the spatial derivatives and solve 
for the x and y optic flow vector.
</li>
<li>Plot the flow vectors on top of the image using <code>quiver</code>.
</li>
<li>Submit a movie of your results (images with overlayed flow 
field). You can use <code>getframe</code> to create a movie. Please make 
sure the movie file is relatively small.
</li>

<pre><code>
Example use of quiver and getframe

X = (1 : blockSize : imageWidth-1) + blockSize/2;
Y = (1 : blockSize : imageHeight-1) + blockSize/2;
[X,Y] = meshgrid(X,Y);
loop through all images (i){
  ...
  loop through all blocks (ix,iy){
    compute motion vectors ...
    U(ix,iy) = xMotion;
    V(ix,iy) = yMotion;
  }
  ...
  plot image;
  hold on;
  quiver(X,Y,U,V);
  hold off;
  M(i) = getframe;
}

movie(M);
save myMovie M;
</code></pre>
</ul>

If you are getting inconsistent results, you could try testing with 
artificial motion first. For example:
<pre><code>
%im1 = image at time 1
%im2 = image at time 2
im2 = zeros(size(im1));
im2(1:end-1, 1:end-1) = im1(2:end, 2:end);
</code></pre>
You may also find it useful to display some of your intermediate 
images/data.
<br clear="all">
<a href="optflow.jpg"><img src="optflow.jpg" width="100"></a>
<br clear="all">
<ul>
<li><strong>Can this optic flow account for any type of motion? If not, give two 
distinct cases when it will not work well.</strong></li>
</ul>
</blockquote>

<b><font size="+1">Exercise 3 (Bonus - 2):</font></b> <font size="+1">High dimensional motion</font>
<blockquote>

Try to extract 3-6 dimensional motion. Parameterize motion using e.g. 4 (x,y,rotation,scale) and 6 (affine) parameters.
How many motion parameters can be reliably extracted? How does the block size affect the results? Try both camera motion and object motion
sequences. Compare results.
<ul>
<li>Each time you may only be able to show 2dof by using quiver. You can show images two
times if you are trying 4 dof, 3 times for 6dof.</li>
<li>While visualizing your extracted motion parameters is difficult, you can also validate your solution by warping the image (or just individual blocks) based on the extracted motion parameters, and checking if the result is more similar to the next frame than if you computed the warped image using your solution to Exercise 2.</li>
</ul>
</blockquote>

<h3>Test sequences (Use as desired — It is not mandatory for you to submit results on them)</h3>
<ul>
<li> How to read and interpret the image sequences. <a href="ReadIm.m">matlab m-
file</a>
</li><li> A 60 image sequence with a jade tree <a href="Flower60im3.mat">Flower60im3.
mat</a>
</li><li> A 32 image sequence with articulated arm movement <a href="armD32im1.mat">a
rmD32im1.mat</a>. This is the same sequence as used in the <a href="armD32im1.mp
g">movie</a> above.
</li></ul>

 